# Initialization

Automatically search for an NVIDIA GPU and use it. If not, then use CPU

In [1]:
import torch
import sys

sys.path.append('/global/u2/q/qingyanz/cgcnn')

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Using GPU')
else:
    device = 'cpu'
    print('Using CPU')

Using GPU


Load all of our preprocessed and split data from our cache

In [2]:
import pickle


with open('../preprocessing/sdt/feature_dimensions.pkl', 'rb') as file_handle:
    orig_atom_fea_len, nbr_fea_len = pickle.load(file_handle)

with open('../preprocessing/splits.pkl', 'rb') as file_handle:
    splits = pickle.load(file_handle)

sdts_train, sdts_val = splits['sdts_train'], splits['sdts_val']
targets_train, targets_val = splits['targets_train'], splits['targets_val']

In [3]:
import sys, os

print(os.path.exists('../preprocessing/feature_dimensions.pkl'))
print(os.path.exists('../preprocessing/splits.pkl'))
print(sys.path)

False
True
['/global/project/projectdirs/m2755/GASpy_workspaces/GASpy/GASpy_regressions', '/global/project/projectdirs/m2755/GASpy_workspaces/GASpy/GASpy_feedback', '/global/project/projectdirs/m2755/GASpy_workspaces/GASpy', '', '/opt/ovis/lib/python2.7/site-packages', '/global/homes/k/ktran/miniconda3/envs/gaspy/lib/python36.zip', '/global/homes/k/ktran/miniconda3/envs/gaspy/lib/python3.6', '/global/homes/k/ktran/miniconda3/envs/gaspy/lib/python3.6/lib-dynload', '/global/homes/k/ktran/miniconda3/envs/gaspy/lib/python3.6/site-packages', '/global/homes/k/ktran/miniconda3/envs/gaspy/lib/python3.6/site-packages/IPython/extensions', '/global/u2/q/qingyanz/.ipython', '/global/u2/q/qingyanz/cgcnn']


Initialize the CGCNN `net` class

In [4]:
from torch.optim import Adam
import skorch.callbacks.base
from skorch import callbacks  # needs skorch >= 0.4  
from skorch import NeuralNetRegressor
from skorch.dataset import CVSplit
from cgcnn.model import CrystalGraphConvNet
from cgcnn.data import collate_pool, MergeDataset


# Callback to checkpoint parameters every time there is a new best for validation loss
cp = callbacks.Checkpoint(monitor='valid_loss_best', fn_prefix='valid_best_')

# Callback to load the checkpoint with the best validation loss at the end of training
class train_end_load_best_valid_loss(callbacks.base.Callback):
    def on_train_end(self, net, X, y):
        net.load_params('valid_best_params.pt')
load_best_valid_loss = train_end_load_best_valid_loss()

# Callback to set the learning rate dynamically
LR_schedule = callbacks.lr_scheduler.LRScheduler('MultiStepLR', milestones=[100], gamma=0.1)

net = NeuralNetRegressor(
    CrystalGraphConvNet,
    module__orig_atom_fea_len=orig_atom_fea_len,
    module__nbr_fea_len=nbr_fea_len,
    batch_size=214,
    module__classification=False,
    lr=0.0056,
    max_epochs=150,
    module__atom_fea_len=46,
    module__h_fea_len=83,
    module__n_conv=8,
    module__n_h=4,
    optimizer=Adam,
    iterator_train__pin_memory=True,
    iterator_train__num_workers=0,
    iterator_train__collate_fn=collate_pool,
    iterator_train__shuffle=True,
    iterator_valid__pin_memory=True,
    iterator_valid__num_workers=0,
    iterator_valid__collate_fn=collate_pool,
    iterator_valid__shuffle=False,
    device=device,
    criterion=torch.nn.L1Loss,
    dataset=MergeDataset,
    callbacks=[cp, load_best_valid_loss, LR_schedule]
)

# Training

We can train a new model...

In [ ]:
net.initialize()
net.fit(sdts_train, targets_train)

Re-initializing module because the following parameters were re-set: atom_fea_len, classification, h_fea_len, n_conv, n_h, nbr_fea_len, orig_atom_fea_len.
Re-initializing optimizer.
  epoch    train_loss    valid_loss    cp      dur
-------  ------------  ------------  ----  -------
      1        0.5684        0.9403     +  21.5256
      2        0.3422        0.5739     +  20.7066
      3        0.2680        0.2394     +  20.6780
      4        0.2883        0.3452        20.7629
      5        0.2523        0.2179     +  20.6928
      6        0.2548        0.2072     +  20.6906
      7        0.2381        0.2266        20.7190
      8        0.2464        0.2329        20.5347
      9        0.2336        0.2880        20.6136
     10        0.2078        0.2067     +  20.6775
     11        0.2160        0.1932     +  20.7319
     12        0.2181        0.1889     +  20.7769
     13        0.2218        0.2256        20.7402
     14        0.2201        0.1894        20.7006
  

...or load whatever is cached

In [ ]:
net.initialize()
net.load_params(f_history='valid_best_history.json',
                f_optimizer= 'valid_best_optimizer.pt', 
                f_params='valid_best_params.pt')

# Assess performance

In [ ]:
%matplotlib inline
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import seaborn as sns


# Calculate the error metrics
targets_pred = net.predict(sdts_val).reshape(-1)
mae = mean_absolute_error(targets_val, targets_pred)
rmse = np.sqrt(mean_squared_error(targets_val, targets_pred))
r2 = r2_score(targets_val, targets_pred)

# Report
print('MAE = %.2f eV' % mae)
print('RMSE = %.2f eV' % rmse)
print('R^2 = %.2f' % r2)

with open('single_cgcnn.pkl', 'wb') as saveplot:
    pickle.dump((targets_pred, mae, rmse, r2), saveplot)

In [ ]:
"""
%matplotlib inline
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import seaborn as sns


# Calculate the error metrics
targets_pred = net.predict(sdts_val).reshape(-1)
mae = mean_absolute_error(targets_val, targets_pred)
rmse = np.sqrt(mean_squared_error(targets_val, targets_pred))
r2 = r2_score(targets_val, targets_pred)

# Report
print('MAE = %.2f eV' % mae)
print('RMSE = %.2f eV' % rmse)
print('R^2 = %.2f' % r2)

# Plot
lims = [-4, 2]
grid = sns.jointplot(targets_val.reshape(-1), targets_pred,
                     kind='hex',
                     bins='log',
                     extent=lims+lims)
_ = grid.ax_joint.set_xlim(lims)
_ = grid.ax_joint.set_ylim(lims)
_ = grid.ax_joint.plot(lims, lims, '--')
_ = grid.ax_joint.set_xlabel('DFT $\Delta$E [eV]')
_ = grid.ax_joint.set_ylabel('CGCNN $\Delta$E [eV]')
"""